In [ ]:
# import sys
# sys.setrecursionlimit(100000)

In [ ]:
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.3 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-z4pbm2yj
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-z4pbm2yj
     |████████████████████████████████| 129 kB 13.9 MB/s 
     |████████████████████████████████| 54.7 MB 17 kB/s 
     |████████████████████████████████| 4.5 MB 41.8 MB/s 
     |████████████████████████████████| 1.2 MB 56.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x39ce4000 @  0x7f4f163d2615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch import optim
import numpy as np

In [ ]:
import gluonnlp as nlp

In [ ]:
from kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

In [ ]:
from tqdm import tqdm, tqdm_notebook

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 설정
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
# 구글드라이브에 있는 파일 가져오기 위해
# 구글드라이브와 연동
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/data/매우긍정_매우부정.xlsx')

In [ ]:
df = df[['word','emo_num']]
df

,word,emo_num
0,가능성이 늘어나다,4
1,가능성이 있다고,1
2,가능하다,0
3,가볍고 상쾌하다,1
4,가볍고 상쾌한,1
...,...,...
7106,골치아프다,15
7107,기진맥진하다,13
7108,꼴불견,11
7109,넌더리나다,11


In [ ]:
df.sample(n=10)

,word,emo_num
4971,성가시게 달라붙어,15
395,깨끗하고,4
3546,매우 괴로워하다,10
5502,쓸데없는 소리를,11
6847,허황되다,10
6114,윽박질하다,6
122,겸손하다,3
4681,불행하게 죽다,6
2255,탐스럽게 피어나는,5
5932,열이 심하게 나고,16


In [ ]:
# df['emotion'].unique()

KeyError: ignored

In [ ]:
# df['emotion']=='분노'

In [ ]:
# list = []
# for i in df['emotion']:
#    list.append(i)

KeyError: ignored

In [ ]:
df.to_excel('/content/drive/MyDrive/data/emotion_sample2.xlsx')

In [ ]:
# df.loc[]

SyntaxError: ignored

In [ ]:
data_list =[]
for i,j in zip(df['word'], df['emo_num']):
  data = []
  data.append(i)
  data.append(str(j))

  data_list.append(data)


In [ ]:
print(data_list[0])

['가능성이 늘어나다', '4']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

5333
1778


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 3942, 7798,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(4, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 4)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# !pip install sentencepiece==0.1.96

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/84 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.061927556991577 train acc 0.046875
epoch 1 train acc 0.46645585317460314


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/28 [00:00<?, ?it/s]

...

epoch 4 test acc 0.72421875


  0%|          | 0/84 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.6372361183166504 train acc 0.84375
epoch 5 train acc 0.8632723922902494


  0%|          | 0/28 [00:00<?, ?it/s]

epoch 5 test acc 0.7279687499999999


In [ ]:
#토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("편안함이")
            elif np.argmax(logits) == 1:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 2:
                test_eval.append("행복이")
            elif np.argmax(logits) == 3:
                test_eval.append("감사가")
            elif np.argmax(logits) == 4:
                test_eval.append("만족이")
            elif np.argmax(logits) == 5:
                test_eval.append("흥미로움이")
            elif np.argmax(logits) == 6:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 7:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 8:
                test_eval.append("당황이")
            elif np.argmax(logits) == 9:
                test_eval.append("상처가")
            elif np.argmax(logits) == 10:
                test_eval.append("불안이")
            elif np.argmax(logits) == 11:
                test_eval.append("짜증이")
            elif np.argmax(logits) == 12:
                test_eval.append("답답이")
            elif np.argmax(logits) == 13:
                test_eval.append("허무함이")
            elif np.argmax(logits) == 14:
                test_eval.append("조롱이")
            elif np.argmax(logits) == 15:
                test_eval.append("불편함이")
            elif np.argmax(logits) == 16:
                test_eval.append("걱정이")
            elif np.argmax(logits) == 17:
                test_eval.append("불만이")
            elif np.argmax(logits) == 18:
                test_eval.append("무서움이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0':
      break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 나는 왕따다
>> 입력하신 내용에서 짜증이 느껴집니다.


하고싶은 말을 입력해주세요 : 나는 왕따를 당한다
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 당한다
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 나는 따다
>> 입력하신 내용에서 조롱이 느껴집니다.


하고싶은 말을 입력해주세요 : 당하
>> 입력하신 내용에서 불만이 느껴집니다.


하고싶은 말을 입력해주세요 : 한다
>> 입력하신 내용에서 답답이 느껴집니다.


하고싶은 말을 입력해주세요 : 나는 괴롭힘을 당한다.
>> 입력하신 내용에서 무서움이 느껴집니다.


하고싶은 말을 입력해주세요 : 나는 왕따를 당한다
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 0
